In [1]:
# Importing the libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [17]:
# Load the dataset
dataset = pd.read_csv(r'input/data/240624_validation_data.csv')


forecast_demand = dataset['predicted heat']